In [ ]:
# install the requirements
!pip install spacy
!python -m spacy download es_core_news_md
!python -m spacy download en_core_web_md

# `spacy`: el *Ruby on Rails* del PLN

[spacy](http://www.spacy.io/) es una librería de procesamiento del lenguaje natural, robusta, rápida, fácil de instalar y utilizar e integrable con [otras librerías de *NLP* y de *deep learning*](https://spacy.io/usage/facts-figures#section-other-libraries). 

Tiene modelos entrenados en varios idiomas y permite realizar las [típicas tareas](https://spacy.io/usage/facts-figures) de segmentación por oraciones, tokenizanción, análisis morfológico, extracción de entidades y análisis de opinión.

Una vez instalados los modelos, podemos importarlos fácilmente:

In [1]:
import spacy

# cargamos el modelo entrenado en español
nlp = spacy.load("es_core_news_md")

In [2]:
texto = """España incumple la regla del déficit del euro y queda como único país bajo el control de Bruselas. 
España no aprobará finalmente la regla europa del déficit y se quedará como único país de la Eurozona que 
suspende y sigue bajo vigilancia. El Gobierno se salta finalmente el requisito de saneamiento presupuestario 
del Tratado de Maastricht y obtiene así más margen de gasto en 2018."""

# y procesamos el texto
doc = nlp(texto)

## Procesando oraciones, palabras y entidades

Podemos iterar fácilmente sobre la lista de oraciones y recorrer los tokens para acceder a su información morfo-sintáctica:

In [3]:
for sentence in doc.sents:
    print("Oración: {}".format(sentence))
    for token in sentence:
        print(
            "{}/{} => etiqueta {}/{} y dependencia {}".format(
                token, token.lemma_, token.pos_, token.tag_, token.dep_
            )
        )

Oración: España incumple la regla del déficit del euro y queda como único país bajo el control de Bruselas. 

España/España => etiqueta PROPN/PROPN y dependencia nsubj
incumple/incumplir => etiqueta VERB/VERB__Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin y dependencia ROOT
la/lo => etiqueta DET/DET__Definite=Def|Gender=Fem|Number=Sing|PronType=Art y dependencia det
regla/reglar => etiqueta NOUN/NOUN__Gender=Fem|Number=Sing y dependencia obj
del/del => etiqueta ADP/ADP__AdpType=Preppron y dependencia case
déficit/déficit => etiqueta NOUN/NOUN__Gender=Masc|Number=Sing y dependencia nmod
del/del => etiqueta ADP/ADP__AdpType=Preppron y dependencia case
euro/euro => etiqueta NOUN/NOUN__Gender=Masc|Number=Sing y dependencia nmod
y/y => etiqueta CCONJ/CCONJ y dependencia cc
queda/quedo => etiqueta VERB/VERB__Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin y dependencia conj
como/comer => etiqueta SCONJ/SCONJ y dependencia mark
único/único => etiqueta ADJ/ADJ__Gender=Masc|Number

El elemeno `doc` tiene una propiedad `.ents` que permite acceder a las entidades nombradas que hayan sido localizadas:

In [4]:
print(doc.ents)

for entity in doc.ents:
    print("{} es de tipo {}".format(entity, entity.label_))

(España, Bruselas, España, Eurozona, El Gobierno, Tratado de Maastricht)
España es de tipo LOC
Bruselas es de tipo LOC
España es de tipo LOC
Eurozona es de tipo ORG
El Gobierno es de tipo LOC
Tratado de Maastricht es de tipo MISC


## Visualizando árboles de dependencias y entidades

Podemos acceder al árbol completo a través del método `.print_tree()`:

In [ ]:
doc.to_json()

Pero también se puede dibujar el grafo con las dependencias:

In [ ]:
from spacy import displacy

displacy.serve(doc, style="dep")

O el de las entidades:

In [5]:
from spacy import displacy

displacy.serve(doc, style="ent")

/home/victor/miniconda3/envs/nlp/lib/python3.8/site-packages/spacy/displacy/__init__.py:94: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [ ]:
texto = """Los ciudadanos españoles son conscientes de que esa salida de Reino Unido no será gratuita. Literalmente. Uno de cada tres teme que el Brexit le cueste caro a la economía española: el 36,9% prevé consecuencias «muy negativas» para sus bolsillos."""
doc = nlp(texto)
displacy.serve(doc, style="ent")

In [6]:
nlp_en = spacy.load("en_core_web_md")

In [7]:
text = """A trade war between the world’s two largest economies officially began on Friday morning as the Trump 
administration followed through with its threat to impose tariffs on $34 billion worth of Chinese products, a 
significant escalation of a fight that could hurt companies and consumers in both the United States and China."""
doc = nlp_en(text)
displacy.serve(doc, style="ent")


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


## Similitud semántica entre palabras, frases y documentos

spaCy permite [calcular la similitud semántica](https://spacy.io/usage/vectors-similarity) entre cualquier par de objetos de tipo `Doc`, `Span` o `Token`. 

Ojo, La similitud semántica es un concepto algo subjetivo, pero en este caso se puede entender como la probabilidad de que dos palabras aparezcan en los mismos contextos.

In [8]:
# analizamos algunas colocaciones en inglés
token1, _, token2 = nlp_en("cats and dogs")
token3, _, token4 = nlp_en("research and development")

# medimos la similitud semántica entre algunos pares
print(token1, "vs", token2, token1.similarity(token2))
print(token3, "vs", token4, token3.similarity(token4))
print(token1, "vs", token4, token1.similarity(token4))

cats vs dogs 0.83117634
research vs development 0.5828718
cats vs development 0.104666404


In [9]:
# ¿qué tal funciona en español?
token1, _, token2 = nlp("perros y gatos")
token3, _, token4 = nlp("investigación y desarrollo")

# medimos la similitud semántica entre algunos pares
print(token1, "vs", token2, token1.similarity(token2))
print(token3, "vs", token4, token3.similarity(token4))
print(token1, "vs", token4, token1.similarity(token4))

perros vs gatos 0.87088835
investigación vs desarrollo 0.69238174
perros vs desarrollo 0.016273146


In [13]:
token2.vector

array([ 2.1515e+00,  1.2182e+00,  7.6666e-02, -5.3475e-01,  3.4992e+00,
        7.4324e-01, -3.8371e+00, -2.4022e+00,  1.4655e+00,  4.3309e+00,
       -2.3522e+00, -7.1208e-01,  2.3551e+00,  5.5128e-01, -1.3499e+00,
       -5.3425e-01, -8.5916e-01, -3.0156e+00,  4.9973e-01, -2.5406e+00,
        3.0369e-02, -9.7725e-01, -4.2862e-02,  4.6845e+00, -1.2609e+00,
       -2.7482e+00,  2.9139e-02, -8.5551e-01, -4.3369e-01,  2.9800e+00,
       -8.1496e-02,  3.6015e+00,  7.1080e-01, -2.8413e-01,  1.9545e-01,
        1.0435e+00,  2.2440e-01, -1.4760e+00,  5.4277e-01,  5.0610e-01,
       -6.5236e-01,  2.7987e+00, -4.3905e+00, -1.1226e+00, -1.3405e+00,
       -1.3589e+00, -4.1786e-01,  2.5365e+00,  2.2368e+00,  1.7861e+00,
        2.3990e+00, -1.3182e-01, -8.0125e-02,  1.5098e+00, -9.7674e-01,
        3.2538e-01,  6.8278e-01,  8.5918e-01, -1.2782e+00,  1.0627e+00,
       -4.3682e-01,  3.4586e-01,  1.3872e+00, -2.3443e+00, -5.5451e-01,
       -4.7095e-01,  1.6325e+00, -2.5439e+00, -1.2369e+00,  1.72